In [1]:
import datetime
import random
import re
import time
from urllib.request import urlopen, Request
import openpyxl
import requests
from bs4 import BeautifulSoup

header = {'User-Agent':
              'Mozilla'}
web_url = r'https://www.imooc.com'
random.seed(datetime.datetime.now().timestamp())

def getLinks(bs):
    return bs.find('div', {'class': 'course-list'}).find_all(lambda tag: len(tag.attrs) == 6)


def cover(bs):
    random.seed(datetime.datetime.now().timestamp())
    cover = bs.findAll('div', {'class': 'img'})
    for c in cover:
        # print(type(c['style']))
        U = re.search(r"url\('?(.*?)'?\)", c['style']).group(1)
        # print(U.split('/')[-1])
        response = requests.get('https:' + U, header)
        time.sleep(random.random())
        filename = "D:\\Developer environment\\Study\\scrapy\\scrapyImage\\" + U.split('/')[-1]
        # urlretrieve('https:' + U,filename) """这也可以下载图片urlretrive(url,filename)"""
        # print(filename)
        with open(filename, 'wb') as f:
            f.write(response.content)

获取外链并爬取每一页的课程图片

In [2]:
externalUrl = []
try:
    for i in range(1, 5):
        W = web_url + '/course/list?ct=1&page=' + str(i)
        req = Request(W, headers=header)
        bs = BeautifulSoup(urlopen(req), 'html.parser')
        links = getLinks(bs)
        cover(bs)

        for i in range(40):
            newArticle = links[random.randint(0, len(links) - 1)].attrs['href']
            externalUrl.append('http:' + newArticle)
except AttributeError:
    pass
print(f'externalUrl size is {len(externalUrl)}.')
print('externalUrl scrapy is Done!')

externalUrl size is 160.
externalUrl scrapy is Done!


In [3]:


def name(bs):
    try:

        name = bs.find('div', {'class': 'hd clearfix'}).text.strip()

    except:
        name = bs.find('div', {'class': 'path'}).find('span').getText()
    return name


def time(bs):
    try:
        time = bs.find('div', {'class': 'info-bar clearfix'}).findAll('span', {'class': 'nodistance'})[1].text
    except AttributeError:
        # print('Error')

        time = bs.findAll('span', {'class': 'meta-value'})[1].text
        # print(f'time is {time}')
    return time


def teacher(bs):
    try:
        teacher = bs.find('div', {'class': 'nickname'}).get_text()
    except:
        # print('Error!')

        teacher = bs.find('span', {'class': 'tit'}).find('a').text
        # print(f'teacher is {teacher}!')
    return teacher


def introduction(bs):
    try:
        introduction = bs.find('div', {'class': 'dec-box'}).get_text()
    except:
        # print('Error')
        introduction = bs.find('div', {"class": "course-description course-wrap"}).text
        # print(f'introduction is {introduction}!')
    return introduction


def price(bs):
    try:
        price = bs.find('div', {'class': 'ori-price l'}).get_text().split()[0]
    except:
        price = '0'
    return price


def address(external_url):
    return external_url


workbook = openpyxl.Workbook()
worksheet = workbook.active
def writeDown(b, e, i):


    worksheet.cell(row=1, column=1, value="name")
    worksheet.cell(row=1, column=2, value="time")
    worksheet.cell(row=1, column=3, value="teacher")
    worksheet.cell(row=1, column=4, value="introduction")
    worksheet.cell(row=1, column=5, value="price")
    worksheet.cell(row=1, column=6, value="address")
    # print('begin')
    # print(f'name is {name(b)}')
    # print(f'time is {time(b)}')
    # print(f'teacher is {teacher(b)}')
    # print(f'introduction is {introduction(b)}')
    # print(f'price is {price(b)}')
    # print(f'address is {address(e)}')
    # print('done')
    worksheet.cell(row=i, column=1, value=name(b))
    worksheet.cell(row=i, column=2, value=time(b))
    worksheet.cell(row=i, column=3, value=teacher(b))
    worksheet.cell(row=i, column=4, value=introduction(b).strip())
    worksheet.cell(row=i, column=5, value=price(b))
    worksheet.cell(row=i, column=6, value=address(e))



print(externalUrl[1])
i = 2
for ext in externalUrl:
    html = urlopen(ext)
    bs = BeautifulSoup(html, 'html.parser')
    writeDown(bs, ext, i)
    i += 1
workbook.save("imooc.xlsx")

http://coding.imooc.com/class/82.html
